In [61]:
### Cell 1: Importing Necessary Libraries
import numpy as np
import random
import math
import matplotlib.pyplot as plt
from typing import List, Tuple


In [62]:
### Cell 2: Town Coordinates
town_coordinates = {
    "Windhoek": (-22.5609, 17.0658),
    "Swakopmund": (-22.6833, 14.5333),
    "Walvis Bay": (-22.9575, 14.5053),
    "Otjiwarongo": (-20.4637, 16.6477),
    "Tsumeb": (-19.2428, 17.7144),
    "Grootfontein": (-19.5667, 18.1167),
    "Mariental": (-24.6333, 17.9667),
    "Keetmanshoop": (-26.5833, 18.1333),
    "Ondangwa": (-17.9167, 15.95),
    "Oshakati": (-17.7833, 15.6833)
}

towns = list(town_coordinates.keys())


distance_matrix = [
    [0, 361, 395, 249, 433, 459, 268, 497, 678, 712],
    [361, 0, 35.5, 379, 562, 589, 541, 859, 808, 779],
    [395, 35.5, 0, 413, 597, 623, 511, 732, 884, 855],
    [249, 379, 413, 0, 260, 183, 519, 768, 514, 485],
    [433, 562, 597, 260, 0, 60, 682, 921, 254, 288],
    [459, 589, 623, 183, 60, 0, 708, 947, 308, 342],
    [268, 541, 511, 519, 682, 708, 0, 231, 909, 981],
    [497, 859, 732, 768, 921, 947, 231, 0, 1175, 1210],
    [678, 808, 884, 514, 254, 308, 909, 1175, 0, 30],
    [712, 779, 855, 485, 288, 342, 981, 1210, 30, 0]
]

In [63]:
### Cell 3: TSP Class
def calculate_distance(route: List[int]) -> float:
    total_distance = 0
    for i in range(len(route)):
        town_a_idx = route[i - 1]
        town_b_idx = route[i]
        total_distance += distance_matrix[town_a_idx][town_b_idx]
    return total_distance

In [64]:
### Cell 4: Simulated Annealing Solver Class
class SimulatedAnnealingSolver:
    def __init__(self, initial_temp: float, cooling_rate: float, max_iterations: int, towns: List[str]):
        self.initial_temp = initial_temp
        self.cooling_rate = cooling_rate
        self.max_iterations = max_iterations
        self.towns = towns


    def random_route(self) -> List[int]:
        # Start with Windhoek (index 0), shuffle the rest
        route = list(range(1, len(self.towns)))  # Exclude Windhoek initially
        random.shuffle(route)
        route = [0] + route  # Prepend Windhoek as the starting point
        return route

    def swap_towns(self, route: List[int]) -> List[int]:
        new_route = route.copy()
        # Ensure we don't swap the first element (Windhoek)
        i, j = random.sample(range(1, len(new_route)), 2)
        new_route[i], new_route[j] = new_route[j], new_route[i]
        return new_route

    def acceptance_probability(self, old_cost: float, new_cost: float, temperature: float) -> float:
        if new_cost < old_cost:
            return 1.0
        return math.exp((old_cost - new_cost) / temperature)

    def solve(self):
        current_route = self.random_route()
        best_route = current_route.copy()
        current_cost = calculate_distance(current_route)
        best_cost = current_cost

        temperature = self.initial_temp
        iteration = 0
        distance_progress = []

        while temperature > 1 and iteration < self.max_iterations:
            new_route = self.swap_towns(current_route)
            new_cost = calculate_distance(new_route)

            if self.acceptance_probability(current_cost, new_cost, temperature) > random.random():
                current_route = new_route
                current_cost = new_cost

            if current_cost < best_cost:
                best_route = current_route.copy()
                best_cost = current_cost

            distance_progress.append(current_cost)
            temperature *= self.cooling_rate
            iteration += 1

        return best_route, best_cost, distance_progress

    def plot_route(self, route: List[int]):
        if not plotting_available:
            print("Matplotlib not available. Cannot plot route.")
            return
            
        route_towns = [towns[i] for i in route] + [towns[route[0]]]
        route_coords = [town_coordinates[town] for town in route_towns]

        x_coords, y_coords = zip(*[(coord[1], coord[0]) for coord in route_coords])

        plt.figure(figsize=(10, 6))
        plt.plot(x_coords, y_coords, marker='o', linestyle='-')
        for i, town in enumerate(route_towns):
            plt.text(x_coords[i], y_coords[i], town, fontsize=9)
        plt.title("Optimized TSP Route")
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.grid()
        plt.show()

    def plot_progress(self, distance_progress: List[float]):
        if not plotting_available:
            print("Matplotlib not available. Cannot plot progress.")
            return
            
        plt.figure(figsize=(10, 6))
        plt.plot(distance_progress, label="Distance")
        plt.title("Simulated Annealing Progress")
        plt.xlabel("Iterations")
        plt.ylabel("Distance (km)")
        plt.legend()
        plt.grid()
        plt.show()


In [65]:
plotting_available = 'matplotlib.pyplot' in globals()

solver = SimulatedAnnealingSolver(initial_temp=1000, cooling_rate=0.995, max_iterations=10000, towns=towns)
best_route, best_cost, distance_progress = solver.solve()

# Print results
print("Initial random route:", [towns[i] for i in solver.random_route()])
print("Best route:", [towns[i] for i in best_route])
print("Total distance (km):", best_cost)

# Plot if matplotlib is available
if plotting_available:
    solver.plot_route(best_route)
    solver.plot_progress(distance_progress)

Initial random route: ['Windhoek', 'Grootfontein', 'Walvis Bay', 'Oshakati', 'Swakopmund', 'Tsumeb', 'Keetmanshoop', 'Ondangwa', 'Mariental', 'Otjiwarongo']
Best route: ['Windhoek', 'Mariental', 'Keetmanshoop', 'Otjiwarongo', 'Grootfontein', 'Tsumeb', 'Oshakati', 'Ondangwa', 'Swakopmund', 'Walvis Bay']
Total distance (km): 3066.5
